In [2]:
import pandas as pd

df = pd.read_csv('datos.csv', delimiter=';')

#print(df.head())

print(df.columns)

Index(['Categoría', 'Enlace', 'Nombre', 'ImagenURL', 'Calorías',
       'Ingredientes', 'Nutrientes', 'Vitaminas y Minerales'],
      dtype='object')


In [3]:
df.isnull().sum()

Categoría                0
Enlace                   0
Nombre                   0
ImagenURL                0
Calorías                 0
Ingredientes             2
Nutrientes               0
Vitaminas y Minerales    3
dtype: int64

In [7]:
df.dropna(axis=0, inplace=True)

In [6]:

print(df['Ingredientes'].iloc[0])
print(df['Nutrientes'].iloc[0])

 | 1 taza de avena tradicional (old fashioned) sin cocer | 1 taza de yogur descremado | 1/2 taza de leche descremada o de 1% de grasa | 1/2 taza de arándanos frescos o congelados | 1/2 taza de trocitos de manzana (aproximadamente 1/3 manzana mediana cortada en 3" de diámetro)
 | Grasa Total 3g | Grasa Saturada 1g | Colesterol 5mg | Sodio 55mg | Carbohidrato Total 27g | Fibra Dietética 3g | Azúcares Totales 9g | Incluye 0g azúcares añadidos | Proteínas 8g


In [8]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Cargar el dataset
#df = pd.read_csv('recetas.csv')

# Limpiar y convertir las calorías a números
df['Calorías'] = pd.to_numeric(df['Calorías'], errors='coerce')

# Limpieza de los ingredientes: separar por " | " y convertir a minúsculas
df['Ingredientes_limpios'] = df['Ingredientes'].apply(
    lambda x: ' '.join([word.lower() for word in str(x).split('|')]) if pd.notna(x) else ''
)

# Vectorización de los ingredientes usando TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['Ingredientes_limpios'])


In [9]:
def extract_calories(prompt):
    match = re.search(r'(\d+)\s*calorías', prompt.lower())
    if match:
        return int(match.group(1))
    else:
        return None


In [20]:
def find_recipe_by_calories_and_ingredients(user_calories, user_prompt):
    # Calculamos la similitud de cosenos entre la receta y las del dataset
    cosine_similarities = cosine_similarity(X)  # Similaridad de ingredientes
    calorie_differences = np.abs(df['Calorías'] - user_calories)  # Diferencia de calorías
    
    # Buscar la receta con la menor diferencia de calorías
    best_match_index = calorie_differences.idxmin()
    
    # Obtener la receta más similar basada en ingredientes y calorías
    best_recipe = df.iloc[best_match_index]
    
    # Mostrar la receta más similar
    return best_recipe

# Ejemplo de entrada del usuario
user_prompt = "quiero una receta que tenga 180 calorías"
user_calories = extract_calories(user_prompt)

if user_calories:
    recipe = find_recipe_by_calories_and_ingredients(user_calories, user_prompt)
    print(f"Receta recomendada: {recipe['Nombre']}, Calorías: {recipe['Calorías']}")
else:
    print("No se encontró información de calorías en tu consulta.")


Receta recomendada: Chili vegetariano, Calorías: 180


In [51]:
import pandas as pd
import spacy
import numpy as np

# Cargar el modelo de Spacy en español
nlp = spacy.load('es_core_news_md')

# Limpieza de los ingredientes: separar por " | " y convertir a minúsculas
df['Ingredientes_limpios'] = df['Ingredientes'].apply(
    lambda x: ' '.join([word.lower() for word in str(x).split('|')]) if pd.notna(x) else ''
)

print(df['Ingredientes_limpios'].iloc[0])



   1 taza de avena tradicional (old fashioned) sin cocer   1 taza de yogur descremado   1/2 taza de leche descremada o de 1% de grasa   1/2 taza de arándanos frescos o congelados   1/2 taza de trocitos de manzana (aproximadamente 1/3 manzana mediana cortada en 3" de diámetro)


In [53]:
import pandas as pd
import spacy
import numpy as np
import re  # Para expresiones regulares

# Cargar el modelo de Spacy en español
nlp = spacy.load('es_core_news_md')

# Función para limpiar los ingredientes y extraer solo los nombres (sin cantidades ni unidades)
def clean_ingredients(ingredient_str):
    # Eliminar cantidades, unidades y texto adicional
    ingredient_str = re.sub(r'\d+\s*(\w+\s?)+', '', ingredient_str)  # Elimina números y unidades
    ingredient_str = re.sub(r'[^\w\s]', '', ingredient_str)  # Eliminar caracteres no alfanuméricos
    return ' '.join([word.lower() for word in ingredient_str.split() if word])

# Limpieza de los ingredientes: separar por " | " y convertir a minúsculas
df['Ingredientes_limpios'] = df['Ingredientes'].apply(
    lambda x: ' '.join([clean_ingredients(ingredient) for ingredient in str(x).split('|')]) if pd.notna(x) else ''
)

# Función para obtener la similitud entre los ingredientes de la receta y el prompt del usuario
def get_similarity(text1, text2):
    # Convertir ambos textos en vectores utilizando Spacy
    vec1 = nlp(text1).vector
    vec2 = nlp(text2).vector
    
    # Verificar si alguna norma es cero (evitar división por cero)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    if norm1 == 0 or norm2 == 0:
        return 0  # Si uno de los vectores es nulo, no tiene sentido calcular similitud
    
    # Calcular la similitud de coseno
    cosine_similarity = np.dot(vec1, vec2) / (norm1 * norm2)
    return cosine_similarity

# Función para buscar recetas por ingredientes
def find_recipe_by_ingredients(user_prompt):
    # Convertir la consulta del usuario a minúsculas
    user_prompt = user_prompt.lower()
    
    # Paso 1: Filtrar recetas que tengan el término en el nombre
    filtered_by_name = df[df['Nombre'].str.contains(user_prompt, case=False, na=False)]
    
    # Si se encuentran recetas que coinciden por nombre, devolver la mejor receta
    if not filtered_by_name.empty:
        return filtered_by_name
    
    # Paso 2: Si no se encuentran coincidencias por nombre, filtrar por ingredientes
    similarities = []
    for ingredientes in df['Ingredientes_limpios']:
        similarity = get_similarity(user_prompt, ingredientes)
        similarities.append(similarity)
    
    # Encontrar la receta con la mayor similitud
    most_similar_index = similarities.index(max(similarities))
    best_recipe = df.iloc[most_similar_index]
    
    return best_recipe

# Ejemplo de entrada del usuario
user_prompt = "Dame una receta que contenga pasta y huevos"
recipe = find_recipe_by_ingredients(user_prompt)

# Mostrar la receta recomendada
if isinstance(recipe, pd.DataFrame):  # Si se encuentran recetas por nombre
    print("Recetas recomendadas por nombre:")
    for _, row in recipe.iterrows():
        print(f"Receta: {row['Nombre']}")
        print(f"Ingredientes: {row['Ingredientes']}")
else:  # Si se encuentra una receta por similitud de ingredientes
    print(f"Receta recomendada: {recipe['Nombre']}")
    print(f"Ingredientes: {recipe['Ingredientes']}")


Receta recomendada: Tazón de arroz al estilo del suroeste
Ingredientes:  | 1 cucharadita de aceite vegetal | 1 taza de vegetales picados (pruebe una mezcla de pimiento dulce, cebolla, maíz, tomate, calabacita) | 1 taza de carne cocinada (picada o deshilachada), frijoles o tofu | 1 taza de arroz integral cocido | 2 cucharadas de salsa, queso rallado o crema agria baja en grasa


In [16]:
print(df.columns)


Index(['Categoría', 'Enlace', 'Nombre', 'ImagenURL', 'Calorías',
       'Ingredientes', 'Nutrientes', 'Vitaminas y Minerales',
       'Ingredientes_limpios'],
      dtype='object')


In [ ]:
import re
import nltk
import spacy
import os
from nltk.corpus import stopwords

nltk.download('stopwords')

directorio_actual = os.getcwd()

try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    spacy.cli.download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

stop_words = set(stopwords.words("spanish"))

def preprocesar_pregunta(pregunta):
   
    pregunta = pregunta.lower()

    pregunta = re.sub(r'[^a-záéíóúüñ\s]', '', pregunta)

    # Tokenizar la pregunta usando spaCy
    doc = nlp(pregunta)

    # Eliminar stopwords y aplicar lematización
    tokens = [token.lemma_ for token in doc if token.text not in stop_words]

   
    pregunta_normalizada = " ".join(tokens)

    return pregunta_normalizada

# TEST
pregunta_ejemplo = "¿Las cachapas llevan queso?"
pregunta_preprocesada = preprocesar_pregunta(pregunta_ejemplo)
print(pregunta_preprocesada)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/edfermachado/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


cuál mejor lugar visitar madrid
